# Setting up

In [ ]:
!apt install swig cmake

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (777 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

In [ ]:
!pip install stable-baselines3[extra]
!pip install huggingface_sb3

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install atari-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.6/540.6 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for atari-py: filename=atari_py-0.2.9-cp310-cp310-linux_x86_64.whl size=2872208 sha256=2050ff6ae70685c3d847a815e154070377f47d4cf60a546a019ee0b30ab12f8e
  Stored in directory: /root/.cache/pip/wheels/75/6f/04/1f3bf5255580101e16ff487564354dddcdd23ec3b43b775b7a
Successfully built atari-py


# Create a virtual display

In [ ]:
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-opengl
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 49 not upgraded.
Need to get 7,815 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Imports


In [ ]:
import gymnasium as gym
import numpy as np
import os
from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
import ale_py
import torch
import torch.nn as nn
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

# Hyperparameter tuning


### Config


In [ ]:
env_id = "SpaceInvadersNoFrameskip-v4"
n_trials = 100 # Maximum number of trials
n_jobs = os.cpu_count() # Number of jobs to run in parallel
n_startup_trials = 5 # Stop random sampling after n_startup_trials
n_evaluations = 10 # Number of evaluations during the training
n_timesteps = int(1e5) # Maximum number of timesteps
eval_freq = int(n_timesteps / n_evaluations)
n_eval_envs = 5
n_eval_episodes = 10
timeout = int(2 * 3600) # 2 hours

default_hyperparams = {
    "policy": "CnnPolicy",
    "env": env_id,
    "buffer_size": 100_000,
}

### Define the search space

In [ ]:
from typing import Any, Dict

def sample_dqn_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Sampler for DQN hyperparameters.
    Args:
        trial: Optuna trial object.
    Returns:
        Dictionary of sampled hyperparameters for the given trial.
    """

    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
        "gamma": trial.suggest_float("gamma", 0.9, 0.999),
        "exploration_fraction": trial.suggest_float("exploration_fraction", 0.1, 0.5),
        "target_update_interval": trial.suggest_int("target_update_interval", 1000, 5000),
        "train_freq": trial.suggest_categorical("train_freq", [2, 4, 8]),
        "gradient_steps": trial.suggest_categorical("gradient_steps", [1, 4, 8]),
        "policy_kwargs": {
            "net_arch": trial.suggest_categorical("net_arch", [[64, 64], [128, 128], [256, 256]]),
        },
    }


### Define the objective function
First we define a custom callback to report the results of periodic evaluations to Optuna.

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.

    :param eval_env: Evaluation environment
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq: Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10_000,
        deterministic: bool = True,
        verbose: int = 0,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

Then we define the objective function that is in charge of sampling hyperparameters, creating the model and then returning the result to Optuna

In [ ]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function used by Optuna to evaluate one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters, evaluate it and report the result
    (mean episodic reward after training)

    Args:
        trial: Optuna trial object
    Returns:
        Mean episodic reward after training
    """

    kwargs = default_hyperparams.copy()
    # Sample parameters
    kwargs.update(sample_dqn_params(trial))

    # Create the RL model
    model = DQN(**kwargs)

    # Create evaluation environment
    eval_env = make_vec_env(env_id, n_envs=n_eval_envs)

    # Create the callback that will periodically evaluate
    eval_callback = TrialEvalCallback(
        eval_env=eval_env,
        trial=trial,
        n_eval_episodes=n_eval_episodes,
        eval_freq=eval_freq,
        verbose=1,
    )

    nan_encountered = False
    try:
        model.learn(n_timesteps, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

### The optimization loop

In [ ]:
sampler = TPESampler(n_startup_trials=n_startup_trials)
# Do not prune before 1/3 of the max budget is used
pruner = MedianPruner(
    n_startup_trials=n_startup_trials, n_warmup_steps=(n_timesteps // 3)
)
# Create the study and start the hyperparameter optimization
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs, timeout=timeout)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Write report
study.trials_dataframe().to_csv("study_results_dqn_spaceInvaders.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

[I 2024-11-20 18:14:07,173] A new study created in memory with name: no-name-d8df3e19-3fd6-4c7a-8189-e3844914f897
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [64, 64] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [256, 256] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/buffers.py:241: Use

# Train the Final Agent

In [ ]:
best_model = DQN(**trial.params)
best_model.learn(int(1e6))
model_name = "dqn-spaceInvadersNoFrameSkip-optuna"
best_model.save(model_name)

# Evaluate the Agent

In [ ]:
eval_env = Monitor(gym.make(env_id))
mean_reward, std_reward = evaluate_policy(best_model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean reward: {mean_reward: .2f}, +/- {std_reward}")

# Publish my trained model on the Huggingface Hub 🔥


In [ ]:
notebook_login()
!git config --global credential.helper store

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv

env_id = "SpaceInvadersNoFrameskip-v4"
model_architecture = "DQN"
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {username}/{repo_name}
repo_id = "wowthecoder/dqn-spaceInvadersNoFrameSkip-optuna"
commit_msg = "Upload DQN Space Invaders agent with tuned hyperparameters from Optuna"
# Create the eval env and set the render mode = "rgb_array"
eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))])

package_to_hub(
    model=best_model,
    model_name=model_name,
    model_architecture=model_architecture,
    env_id=env_id,
    eval_env = eval_env,
    repo_id = repo_id,
    commit_message=commit_msg,
)